In [ ]:
%matplotlib inline
import pandas as pd 
import numpy as np

import torch 
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder


device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")


from modules.classifer_dataset import ClassifierDataset


In [ ]:
# Load the Titanic dataset.
titanic_train_csv_df = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_test_csv_df = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")

print(f'titanic_train_csv_df shape: {titanic_train_csv_df.shape}')
print(f'titanic_test_csv_df shape: {titanic_test_csv_df.shape}')

# concat test and train to get metadata for non numeric categories
csv_union_df = pd.concat([titanic_test_csv_df, titanic_train_csv_df])



In [ ]:

label_column_name = "survived"
[train_encoded_df, test_encoded_df] = ClassifierDataset.onehot_encode_datafames([titanic_train_csv_df, titanic_test_csv_df])


train_ds = ClassifierDataset(train_encoded_df, label_column_name)
test_ds = ClassifierDataset(test_encoded_df, label_column_name)

batch_size = int(len(train_ds) / 10)

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, shuffle=True)
print(f'{len(train_ds)} training records in with batch size {batch_size}, {len(test_ds)} records for test')

print(f'train has {train_ds.get_feature_count()} features')
print(f'test has {test_ds.get_feature_count()} features')




In [ ]:
DROPOUT_RATE_01 = .20

# TODO have a config for layers and wire them from it

class TitanicSurvivalNeuralNetwork(nn.Module):
    def __init__(self, num_feature_columns):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(num_feature_columns, 64),
            nn.GELU(),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.Linear(64, 64),
            nn.GELU(),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.Linear(64, 32),
            nn.GELU(),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.Linear(32, 32),
            nn.GELU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.linear_relu_stack(x)


first_training_record, _ = train_ds[0]
num_features = first_training_record.shape[-1]
model = TitanicSurvivalNeuralNetwork( num_features)
print(model)

In [ ]:


SEED = 123
torch.manual_seed(SEED)


if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)


num_features = train_ds.get_feature_count()
model = TitanicSurvivalNeuralNetwork( num_features )
model.to(device)

loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    epoch_correct_count, epoch_pred_count = 0, 0
    for X, y in train_dataloader:

        X = X.to(device)
        y = y.reshape(-1, 1).to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        y_pred_guess = torch.round(y_pred)
        batch_num_correct = (y == y_pred_guess).sum()
        epoch_correct_count += batch_num_correct
        epoch_pred_count += len(y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], {epoch_correct_count} of {epoch_pred_count} correct {(100*epoch_correct_count/epoch_pred_count):.1f} %")


In [ ]:

model.eval()  # Set the model to evaluation mode

# Define your loss function (e.g., Binary Cross-Entropy)
criterion = nn.BCEWithLogitsLoss()  # Commonly used for binary classification

# Lists to store results
test_losses = []
all_preds = []
all_labels = []

# Iterate over the test batches
with torch.no_grad():  # Disable gradient calculations
    for inputs, labels in test_dataloader:
        # just make this an array of the labels (instead array of arrays with one element)
        labels = labels.reshape(-1)

        y_pred = model(inputs)
        y_pred = y_pred.reshape(labels.shape)  # make sure it matches

        loss = criterion(y_pred, labels)
        test_losses.append(loss.item())

        y_pred_guess = torch.round(y_pred)
        
        all_preds.extend(y_pred_guess.numpy())
        all_labels.extend(labels.numpy())

# Calculate overall metrics
avg_test_loss = np.mean(test_losses)
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Average Test Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")